# Set the environment

In [1]:
# set the envoiroment
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import escher
from escher import Builder
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem


## load the model

In [2]:
model = cobra.io.load_json_model("../models/updated_model.json")# Import the model.
model.solver = 'glpk'
model.reactions.NGAM.bounds = (0,1000)
model.objective = model.reactions.NGAM
map_loc = "../data/overallmap.json"
map_loc1 = "../data/7.21 overallmap.json" #Import the map that has been drawn using escher.

## PK

### PGK and GAPDH pathway + PK + SoxM: Figure S4

In [3]:
M= model.copy()
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])

b = show_map(sol,map_loc1)
b

ATP production flux: 8.000000000000002


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### PGK and GAPDH pathway + PK + SoxABCD

In [4]:
M= model.copy()
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 14.0


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [5]:
model.reactions.get_by_id('cof_sulfopterin_6.3.4.3_FORMATETHFLIG__RXN').bounds = (0,0)

### PGK and GAPDH pathway + PK + SoxM

In [6]:
M= model.copy()
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 8.000000000000005


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Sox ABCD + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [7]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 12.999999999999982


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Sox M + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [8]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 7.000000000000004


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxABCD + spontaneous degradation pathway after blocking the reaction for GK

In [9]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 13.000000000000005


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM + spontaneous degradation pathway after blocking the reaction for GK

In [10]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 7.000000000000004


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxABCD + GAPN pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [11]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 12.999999999999996


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM + GAPN pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [12]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 7.000000000000004


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxABCD+ methyxol

In [13]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN').bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 10.000000000000005


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM + methyxol

In [14]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 5.000000000000008


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## PEP carboxlyse + PEP synthesis:  pathway 2

### SoxABCD PGK and GAPDH pathway + SoxABCD

In [15]:
model.reactions.get_by_id ("aa_ser_gly_cys_sec_trp_1.1.1.95_PGLYCDEHYDROG__RXN").bounds = (0,0)

In [16]:
M= model.copy()
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 13.000000000000028


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM PGK and GAPDH pathway + PK + SoxM

In [17]:
M= model.copy()
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 7.000000000000003


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Sox ABCD + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [18]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 11.999999999999995


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Sox M + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [19]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 5.999999999999993


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxABCD + spontaneous degradation pathway after blocking the reaction for GK

In [20]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 12.000000000000018


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM + spontaneous degradation pathway after blocking the reaction for GK

In [21]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 6.0000000000000195


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxABCD + GAPN pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [22]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 12.00000000000001


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM + GAPN pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [23]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
# for rxn in l:
#     print (rxn,M.reactions.get_by_id(rxn).reaction,sol.fluxes.loc[rxn])
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 5.999999999999984


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## blocking PEP carboxlyse + PEP synthesis:  pathway 3

### SoxABCD PGK and GAPDH pathway + SoxABCD

In [24]:
reaction = Reaction('aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN')
reaction.name = 'aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"L__serine":-1.0,
                         "pyruvate":1.0,
                         "ammonia": 1.0})
model.reactions.get_by_id('aa_met_salvage_4.2.1.22_CYSTATHIONINE__BETA__SYNTHASE__RXN').bounds = (0,0)# these two reactions do not exist in this organism

In [25]:
model.reactions.get_by_id("carb_anap_4.1.1.31_PEPCARBOX__RXN").bounds = (0,0)
model.reactions.get_by_id("aa_ser_gly_cys_sec_trp_1.1.1.95_PGLYCDEHYDROG__RXN").bounds = (0,10)

In [26]:
M= model.copy()
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 13.000000000000025


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM PGK and GAPDH pathway + PK + SoxM

In [27]:
M= model.copy()
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 7.000000000000003


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Sox ABCD + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [28]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 11.999999999999998


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Sox M + Non-phosphorylation pathway after blocking the reactions for KDGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [29]:
M= model.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 6.000000000000001


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxABCD + GAPN pathway after blocking the reaction for GK

In [30]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 12.000000000000002


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxM + GAPN pathway after blocking the reaction for GK

In [31]:
M= model.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

ATP production flux: 6.0000000000000036


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### SoxABCD + spontaneous degradation pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [ ]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)
sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

### SoxM + spontaneous degradation pathway after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [ ]:
M= model.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id("oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240").bounds = (0,0)
M.reactions.get_by_id("carb_enter_2.7.1.40_PEPDEPHOS__RXN").bounds = (0,0)

sol = M.optimize()
print ("ATP production flux:", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc)
b

# Methodology for dealing with variable stoichiometry

In [ ]:
df_variable_stoichiometry = pd.DataFrame(columns=["Gear Z","SoxABCD flux","SoxM (DoxABCD) flux","NGAM (maintenance ATPase) (ATP/glucose imported","H+-ATPsynthase proton flux per glucose imported"]) # start empty table
M = model.copy()
M.objective = M.reactions.NGAM
M.reactions.T_flux_beta__D__glucose.bounds = (-1,1000)
for Z in range(0,13):
    M.reactions.get_by_id('oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240').bounds =(0,Z) 
    M.reactions.get_by_id('oxp_redox_1.10.3.13_soxM_TRANS__RXN__240').bounds =(0,12-Z) 
    sol = M.optimize()
    H_ATPsynthase_proton_flux = sol.fluxes.loc["oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240"] * 4 + sol.fluxes.loc["oxp_redox_1.10.3.13_soxM_TRANS__RXN__240"]*2
    df_variable_stoichiometry.loc[Z] = [Z,sol.fluxes.loc["oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240"],sol.fluxes.loc["oxp_redox_1.10.3.13_soxM_TRANS__RXN__240"],sol.fluxes.loc["NGAM"],H_ATPsynthase_proton_flux] 
# df_variable_stoichiometry = df_variable_stoichiometry.set_index('Factor Z')
df_variable_stoichiometry
df_variable_stoichiometry.to_csv("../tables/Table 3.csv")

In [ ]:
df_variable_stoichiometry